In [1]:

import pandas as pd
from pathlib import Path

excel_path = Path(r"/Users/cheickberthe/PycharmProjects/spreadsheet-analyzer/test_assets/collection/business-accounting/Business Accounting.xlsx")
df = pd.read_excel(excel_path, sheet_name=7)
print(f"Loaded sheet with shape: {df.shape}")

# Quick preview for LLM context
print("\nFirst 10 rows:")
print(df.head(10).to_string())
print("\nLast 5 rows:")
print(df.tail(5).to_string())

# Get basic info
sheet_name = "None"
sheet_dimensions = f"{df.shape[0]} rows x {df.shape[1]} columns"


Loaded sheet with shape: (38, 9)

First 10 rows:
                 Unnamed: 0                 Carosserie Line Unnamed: 2            2 x 20ft Line Unnamed: 4         40-45ft Line Unnamed: 6 Estimated monthly revenue Estimated monthly profit
0                       NaN                     BKO-Sikasso    1100000  containers dakar-bamako    2300000               gazoil    -906000                  13880000                  -300000
1                       NaN  sikasso-bamako-dakar carburant    -858000                Condition     -50000  container dakar-bko    1500000                       NaN                      NaN
2                       NaN                  frais de route    -125000                   Gazoil   -1076000                Prime     -15000          Monthly salaries                      NaN
3                       NaN                riz dakar-bamako    2040000           Frais de route    -150000       Frais de route    -150000                   -300000                      NaN
4

In [2]:

excel_file_name = excel_path.name
sheet_dimensions = f"{df.shape[0]} rows x {df.shape[1]} columns"
sheet_name


'None'

In [3]:

# Table Detection Analysis
print("=== DATA STRUCTURE ANALYSIS ===")
print(f"DataFrame shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\n=== FIRST 10 ROWS ===")
print(df.head(10).to_string())
print("\n=== LAST 5 ROWS ===")
print(df.tail(5).to_string())

print("\n=== COLUMN ANALYSIS ===")
# Check for empty columns that might separate tables
for i, col in enumerate(df.columns):
    non_null_count = df[col].notna().sum()
    print(f"Column {i} ('{col}'): {non_null_count}/{len(df)} non-null values")

print("\n=== EMPTY ROW ANALYSIS ===")
# Check for empty rows that might separate tables
empty_rows = df.isnull().all(axis=1)
if empty_rows.any():
    empty_row_indices = empty_rows[empty_rows].index.tolist()
    print(f"Empty rows found at indices: {empty_row_indices[:10]}")
else:
    print("No completely empty rows found")

print("\n=== SIDE-BY-SIDE ANALYSIS ===")
# Check for potential side-by-side tables by examining column groups
left_cols = df.iloc[:, 0:7]  # First 7 columns
right_cols = df.iloc[:, 7:]  # Remaining columns
print(f"Left section shape: {left_cols.shape}")
print(f"Right section shape: {right_cols.shape}")
print(f"Left section non-null density: {left_cols.notna().sum().sum() / (left_cols.shape[0] * left_cols.shape[1]):.2f}")
print(f"Right section non-null density: {right_cols.notna().sum().sum() / (right_cols.shape[0] * right_cols.shape[1]):.2f}")


=== DATA STRUCTURE ANALYSIS ===
DataFrame shape: (38, 9)
Columns: ['Unnamed: 0', 'Carosserie Line', 'Unnamed: 2', '2 x 20ft Line', 'Unnamed: 4', '40-45ft Line', 'Unnamed: 6', 'Estimated monthly revenue', 'Estimated monthly profit']

=== FIRST 10 ROWS ===
                 Unnamed: 0                 Carosserie Line Unnamed: 2            2 x 20ft Line Unnamed: 4         40-45ft Line Unnamed: 6 Estimated monthly revenue Estimated monthly profit
0                       NaN                     BKO-Sikasso    1100000  containers dakar-bamako    2300000               gazoil    -906000                  13880000                  -300000
1                       NaN  sikasso-bamako-dakar carburant    -858000                Condition     -50000  container dakar-bko    1500000                       NaN                      NaN
2                       NaN                  frais de route    -125000                   Gazoil   -1076000                Prime     -15000          Monthly salaries           

In [4]:
detected_tables = []

# Based on the analysis, the data appears to contain two main sections side-by-side.
# Left section likely contains detailed data, while the right section contains summary or additional info.

# Detecting the main left table (columns 0-6)
# It spans from row 0 to 37, with some empty rows at 7, 12, 30, but these are within the table boundary.
left_table = {
    "start_row": 0,
    "end_row": 37,
    "start_col": 0,
    "end_col": 6,
    "description": "Main detailed data with various costs and revenues",
    "entity_type": "business costs and revenues",
    "confidence": 0.9,
    "table_type": "DETAIL"
}

# Detecting the secondary table (columns 7-8)
# It appears to be a summary or additional info, possibly at the bottom, but since it overlaps with main data, treat as a separate table at the bottom.
# The last few rows (33-37) contain totals and summaries.
secondary_table = {
    "start_row": 33,
    "end_row": 37,
    "start_col": 7,
    "end_col": 8,
    "description": "Summary totals and calculations",
    "entity_type": "summary",
    "confidence": 0.85,
    "table_type": "SUMMARY"
}

detected_tables = [left_table, secondary_table]


In [5]:

# Extract detection results created by LLM
# The detector should have created a 'detected_tables' variable with the results

# CLAUDE-TEST-WORKAROUND: Validate that detected_tables exists and is properly formatted
if 'detected_tables' in globals():
    # Validate it's a list
    if isinstance(detected_tables, list):
        detection_results = detected_tables
        print(f"✅ Found {len(detection_results)} tables from LLM detection")
        # Validate first table has required fields (if any tables exist)
        if detection_results:
            required_fields = ['table_id', 'description', 'start_row', 'end_row', 'start_col', 'end_col']
            first_table = detection_results[0]
            missing_fields = [f for f in required_fields if f not in first_table]
            if missing_fields:
                print(f"⚠️ Warning: First table missing fields: {missing_fields}")
    else:
        print(f"❌ Error: detected_tables is not a list, it's a {type(detected_tables)}")
        detection_results = []
else:
    # CLAUDE-GOTCHA: Gemini sometimes fails to create the variable even after multiple prompts
    print("❌ No 'detected_tables' variable found - LLM failed to complete detection")
    detection_results = []

detection_results


✅ Found 2 tables from LLM detection
⚠️ Warning: First table missing fields: ['table_id']


[{'start_row': 0,
  'end_row': 37,
  'start_col': 0,
  'end_col': 6,
  'description': 'Main detailed data with various costs and revenues',
  'entity_type': 'business costs and revenues',
  'confidence': 0.9,
  'table_type': 'DETAIL'},
 {'start_row': 33,
  'end_row': 37,
  'start_col': 7,
  'end_col': 8,
  'description': 'Summary totals and calculations',
  'entity_type': 'summary',
  'confidence': 0.85,
  'table_type': 'SUMMARY'}]


# Table Detection Results

Detected 2 tables:


## Table 1: Main detailed data with various costs and revenues
- Location: Rows 0-37, Columns 0-6
- Type: detail
- Entity: business costs and revenues
- Confidence: 0.90

## Table 2: Summary totals and calculations
- Location: Rows 33-37, Columns 7-8
- Type: summary
- Entity: summary
- Confidence: 0.85
